In [1]:
import pandas as pd
import numpy as np
import operator
import re
import sys
import os

# Extract Features From MATCH / NON-MATCH

In [2]:
def extract_normal_features(path_in, path_normal): 
    m = pd.read_csv(path_in)

    # calculate the features
    # question_id :
    question_id = m['question_id']
    question_id.to_csv(path_normal+'/qid.txt')

    # vote :
    vote = m['answer_voted']
    vote.to_csv(path_normal+'/vote.txt')

    # feature1 : length of the answer_content 
    #answer_content = m[['answer_content','user_id']]
    answer_content = m['answer_content']
    # f1 : length of the answer_content
    f1 = answer_content.str.len()
    f1.to_csv(path_normal+'/f1.txt')
    #print answer_content


    # feature2 : length of the user_introduction
    user_intro = m['user_intro']
    #print user_intro
    f2 = user_intro.str.len()
    f2.to_csv(path_normal+'/f2.txt')
    #print f2


    # feature3 : length of the question_content
    question_content = m['question_content']
    f3 = question_content.str.len()
    #print f3
    f3.to_csv(path_normal+'/f3.txt')


    # feature4: user's experience years
    user_exp = m['user_exp']
    f4 = m['user_exp']
    f4 = f4.str.extract('([0-9]*)')
    f4.to_csv(path_normal+'/f4.txt')
    #print f4

    # feature5 : length of the user's edu
    user_edu = m['user_edu']
    f5 = user_edu.str.len()
    f5.to_csv(path_normal+'/f5.txt')
    #print f5


    # feature6 : number of topics of user's interest
    user_interest = m['user_interest'].str.split(' ')
    f6 = user_interest.str.len()
    f6.to_csv(path_normal+'/f6.txt')
    #print f6


    # feature7 : number of the people user followed
    user_followed = m['user_followed'].str.split(',')
    f7 = user_followed.str.len()
    f7.to_csv(path_normal+'/f7.txt')
    #print f7

    # feature8 : number of the question tags
    question_tags = m['question_tags'].str.split(' ')
    f8 = question_tags.str.len()
    f8.to_csv(path_normal+'/f8.txt')
    #print f8


    # feature9 : number of saved lives
    saved_lives = m['user_saved']
    f9 = saved_lives
    f9.to_csv(path_normal+'/f9.txt')
    #print f9


    # feature10 : number of recieved thanks
    recieved_thanks = m['user_thanks']
    #recieved_thanks = str(recieved_thanks)
    f10 = recieved_thanks.str.replace(',','')
    f10.to_csv(path_normal+'/f10.txt')
    #print f10


    # feature11 : number of agrees
    recieved_agrees = m['user_agrees']
    f11 = recieved_agrees.str.replace(',','')
    f11.to_csv(path_normal+'/f11.txt')
    #print f11


    # feature12 : nuber of helped people
    helped = m['user_helped']
    f12 = helped.str.replace(',','')
    f12.to_csv(path_normal+'/f12.txt')
    #print f12


    # feature13 : number of doctor recommend
    recommend = m['user_recommends']
    f13 = recommend.str.len()
    f13.to_csv(path_normal+'/f13.txt')
    #print f13


# Extracting for the TEXT FEATRUES
def extract_text_features(path_in, path_text):
    
    m = pd.read_csv(path_in)

    # Process with the question_content
    qc = m['question_content']
    qc.to_csv(path_text+'/question_content.txt', encoding='utf-8')

    # Process with the answer_content
    ac = m['answer_content']
    ac.to_csv(path_text+'/answer_content.txt', encoding='utf-8')

    # Process with the user_recommends
    ur = m['user_recommends'].astype(str)
    ur.to_csv(path_text+'/user_recommends.txt', encoding='utf-8')

    # Process with question_tags
    qt = m['question_tags'].astype(str)
    qt.to_csv(path_text+'/question_tags.txt', encoding='utf-8')

    # Process with user_interest
    ut = m['user_interest'].astype(str)
    ut.to_csv(path_text+'/user_interest.txt', encoding='utf-8')

    # Proecess with user_specialty
    us = m['user_specialty'].astype(str)
    us.to_csv(path_text+'/user_specialty.txt', encoding='utf-8')

In [3]:
from itertools import izip

def unique(a):
    return list(set(a))

def intersect(a,b):
    return list(set(a)&set(b))

def union(a,b):
    return list(set(a) | set(b))

def write_overlap_file(fout, overlap_list, count):
    num_overlap = len(overlap_list) 
    fout.write(str(count)+',')
    fout.write(' '.join(overlap_list))
    fout.write(','+str(num_overlap)+'\n')   

    
# Run this code for best_answer_features
# PATH_IN = 'DIVIDE/match/best_answer_features/OVER_LAP/'
# PATH_OUT = 'DIVIDE/match/best_answer_features/OVER_LAP/'
# PATH_FEATURES = 'DIVIDE/match/best_answer_features/'

def extract_overlap_features(PATH_TEXT, PATH_OVERLAP, PATH_NORMAL) :
    
    f14 = open(PATH_NORMAL + 'f14.txt', 'w')
    f15 = open(PATH_NORMAL + 'f15.txt', 'w')
    f16 = open(PATH_NORMAL + 'f16.txt', 'w')
    f17 = open(PATH_NORMAL + 'f17.txt', 'w')
    f18 = open(PATH_NORMAL + 'f18.txt', 'w')
    f19 = open(PATH_NORMAL + 'f19.txt', 'w')
    
    with open(PATH_TEXT + 'answer_content.txt' , 'r') as fin_1, \
        open(PATH_TEXT + 'question_content.txt', 'r') as fin_2, \
        open(PATH_TEXT + 'question_tags.txt' , 'r') as fin_3, \
        open(PATH_TEXT + 'user_interest.txt' , 'r') as fin_4, \
        open(PATH_OVERLAP + 'answerContent_questionContent.txt', 'w') as fout_1, \
        open(PATH_OVERLAP + 'answerContent_questionTags.txt', 'w') as fout_2, \
        open(PATH_OVERLAP + 'answerContent_userInterest.txt', 'w') as fout_3, \
        open(PATH_OVERLAP + 'questionContent_questionTags.txt', 'w') as fout_4, \
        open(PATH_OVERLAP + 'questionContent_userInterest.txt', 'w') as fout_5, \
        open(PATH_OVERLAP + 'questionTags_userInterest.txt', 'w') as fout_7 : 
            
            count = 0

            for line1, line2, line3, line4 in izip(fin_1, fin_2, fin_3, fin_4 ):
                word_list1 = line1.strip().split(' ')
                word_list2 = line2.strip().split(' ')
                word_list3 = line3.strip().split(' ')
                word_list4 = line4.strip().split(' ')
                
                over_lap1 = intersect(word_list1, word_list2)
                over_lap2 = intersect(word_list1, word_list3)
                over_lap3 = intersect(word_list1, word_list4)
                over_lap4 = intersect(word_list2, word_list3)
                over_lap5 = intersect(word_list2, word_list4)
                # over_lap6 = intersect(word_list2, word_list5)
                over_lap7 = intersect(word_list3, word_list4)
                # over_lap8 = intersect(word_list3, word_list5)
                
                # '''
                write_overlap_file(fout_1, over_lap1, count)
                write_overlap_file(fout_2, over_lap2, count)
                write_overlap_file(fout_3, over_lap3, count)
                write_overlap_file(fout_4, over_lap4, count)
                write_overlap_file(fout_5, over_lap5, count)
                # write_overlap_file(fout_6, over_lap6, count)
                write_overlap_file(fout_7, over_lap7, count)
                # write_overlap_file(fout_8, over_lap8, count)
               
                f14.write(str(count) + ',' + str( len(over_lap1) ) + '\n')
                f15.write(str(count) + ',' + str( len(over_lap2) ) + '\n')
                f16.write(str(count) + ',' + str( len(over_lap3) ) + '\n')
                f17.write(str(count) + ',' + str( len(over_lap4) ) + '\n')
                f18.write(str(count) + ',' + str( len(over_lap5) ) + '\n')
                f19.write(str(count) + ',' + str( len(over_lap7) ) + '\n')
                
                count = count + 1
    
    f14.close()
    f15.close()
    f16.close()
    f17.close()
    f18.close()
    f19.close()

In [4]:
def lda_source(PATH_TEXT, PATH_LDA) :
    
    with open(PATH_TEXT + 'answer_content.txt' , 'r') as fin_1, \
        open(PATH_TEXT + 'question_content.txt', 'r') as fin_2, \
        open(PATH_TEXT + 'user_interest.txt' , 'r') as fin_3, \
        open(PATH_TEXT + 'question_tags.txt' , 'r') as fin_4, \
        open(PATH_LDA + 'LDA_SOURCE.txt', 'w') as fout :

            for line1, line2, line3, line4 in izip(fin_1, fin_2, fin_3, fin_4) :
                #line1 = f1.readline()
                info1 = line1.strip()
                # print info1
                fout.write(info1)

                #line2 = f2.readline()
                info2 = line2.strip()
                # print info2
                fout.write(' '+info2)

                info3 = line3.strip()
                # print info3
                fout.write(' '+info3)

                info4 = line4.strip()
                # print info4
                fout.write(' '+info4 + '\n')

In [5]:
def normalize( PATH_NORMAL ) :
    # feature1 : length of the answer_content 
    f1 = pd.read_csv(PATH_NORMAL + 'f1.txt', names = ['index','len_answer_content'])

    # feature2 : length of the user_introduction
    f2 = pd.read_csv(PATH_NORMAL + 'f2.txt', names = ['index','len_user_intro'])
    m = pd.merge(f1, f2, how='outer')
    #print m

    # feature3 : length of the question_content
    f3 = pd.read_csv(PATH_NORMAL + 'f3.txt', names = ['index', 'len_question_content'])
    m = pd.merge(m, f3, how='outer')

    # feature4: user's experience years
    f4 = pd.read_csv(PATH_NORMAL + 'f4.txt', names = ['index','user_exp'])
    m = pd.merge(m, f4, how='outer')
    #print m

    # feature5 : length of the user's edu
    f5 = pd.read_csv(PATH_NORMAL + 'f5.txt', names = ['index','len_user_edu'])
    m = pd.merge(m, f5, how='outer')

    # feature6 : number of topics of user's interest
    f6 = pd.read_csv(PATH_NORMAL + 'f6.txt', names = ['index','num_user_interst'])
    m = pd.merge(m, f6, how='outer')

    # feature7 : number of the people user followed
    f7 = pd.read_csv(PATH_NORMAL + 'f7.txt', names = ['index','num_user_followed'])
    m = pd.merge(m, f7, how='outer')

    # feature8 : number of the question tags
    f8 = pd.read_csv(PATH_NORMAL + 'f8.txt', names = ['index', 'num_question_tags'])
    m = pd.merge(m, f8, how='outer')

    # feature9 : number of saved lives
    f9 = pd.read_csv(PATH_NORMAL + 'f9.txt', names = ['index','num_user_saved'])
    m = pd.merge(m, f9, how='outer')

    # feature10 : number of recieved thanks
    f10 = pd.read_csv(PATH_NORMAL + 'f10.txt', names = ['index','num_user_thanks'])
    m = pd.merge(m, f10, how='outer')

    # feature11 : number of agrees
    f11 = pd.read_csv(PATH_NORMAL + 'f11.txt', names = ['index','num_user_agrees'])
    m = pd.merge(m, f11, how='outer')

    # feature12 : nuber of helped people
    f12 = pd.read_csv(PATH_NORMAL + 'f12.txt', names = ['index','num_user_helped'])
    m = pd.merge(m, f12, how='outer')

    # feature13 : number of doctor recommend
    f13 = pd.read_csv(PATH_NORMAL + 'f13.txt', names = ['index','num_user_recommend'])
    m = pd.merge(m, f13, how='outer')

    #####################################################################
    
    # feature14 : answerContent & questionContent overlap 
    f14 = pd.read_csv(PATH_NORMAL + 'f14.txt', names = ['index', 'ac_qc_overlap'] )
    m = pd.merge(m, f14, how = 'outer')
    
    # feature15 : answerContent & questionTags overlap
    f15 = pd.read_csv(PATH_NORMAL + 'f15.txt', names = ['index', 'ac_qt_overlap'])
    m = pd.merge(m, f15, how = 'outer')

    # feature16 : answerContent & userInterest overlap
    f16 = pd.read_csv(PATH_NORMAL + 'f16.txt', names = ['index', 'ac_ui_overlap'])
    m = pd.merge(m, f16, how = 'outer')

    # feature17 : questionContent & questionTags overlap
    f17 = pd.read_csv(PATH_NORMAL + 'f17.txt', names = ['index', 'qc_qt_overlap'])
    m = pd.merge(m, f17, how = 'outer')

    # feature18 : questionContent & userInterest overlap
    f18 = pd.read_csv(PATH_NORMAL + 'f18.txt', names = ['index', 'qc_ui_overlap'])
    m = pd.merge(m, f18, how = 'outer')
    
    # feature19 : questionTags & userInterest overlap
    f19 = pd.read_csv(PATH_NORMAL + 'f19.txt', names = ['index', 'qt_ui_overlap'])
    m = pd.merge(m, f19, how = 'outer')
    
    #######################################################################
    Index = m.ix[:,0] 
    M = m.ix[:,1:]

    M = (M - M.min()) / (M.max() - M.min())
    M['index'] = Index

    # question_id :
    # qid = pd.read_csv(PATH_IN + 'qid.txt', names = ['index', 'question_id'])
    # M = pd.merge(M, qid, how = 'outer')

    # number_of_vote
    # vote = pd.read_csv(PATH_IN + 'vote.txt', names = ['index', 'vote'])
    # M = pd.merge(M, vote, how = 'outer')

    M = M.drop('index', axis = 1)
    M = M.fillna(0)
    # print M

    M.to_csv(PATH_NORMAL + 'Normal_Features.txt',  index = False)

In [6]:
PATH_IN = './Final_Test_Set.csv'
PATH_NORMAL = './NORMAL/'
PATH_TEXT = './TEXT/'
PATH_OVERLAP = './OVERLAP/'
PATH_LDA = './LDA/'

extract_normal_features(PATH_IN, PATH_NORMAL)
extract_text_features(PATH_IN, PATH_TEXT)
extract_overlap_features(PATH_TEXT, PATH_OVERLAP, PATH_NORMAL)
lda_source(PATH_TEXT, PATH_LDA)
normalize(PATH_NORMAL)

# Format S2V & Topic Features

In [7]:
# format s2v features :
s2v_list = []
for i in range(50) :
    col_name = 's2v_' + str(i) 
    s2v_list.append(col_name)
# !!! before this , you should remove the first row of the Test_Source.txt.vec
total_s2v = pd.read_table('./SENTENCE_VECTOR/total_s2v.txt.vec', sep = ' ', names = s2v_list)
# test_s2v !!! very important : Normalization
total_s2v = (total_s2v - total_s2v.min()) / (total_s2v.max() - total_s2v.min())
# total_s2v # 11292 rows × 50 columns
total_s2v.to_csv('./SENTENCE_VECTOR/Total_S2V.csv', encoding='utf-8', index = False)

In [8]:
# format the topic features
topic_list = []
for i in range(50) :
    col_name = 'topic_' + str(i) 
    topic_list.append(col_name)

total_topic = pd.read_table('./LDA/model-final.theta', sep = ' ', names = topic_list, index_col=False)
# test_topic !!! very important : Normalization
total_topic = (total_topic - total_topic.min()) / (total_topic.max() - total_topic.min())
total_topic.to_csv('./LDA/Total_Topic.csv', encoding='utf-8', index=False)

# Merge 3 types Features

In [9]:
Total_Normal = pd.read_csv('./NORMAL/Normal_Features.txt')
Total_S2V = pd.read_csv('./SENTENCE_VECTOR/Total_S2V.csv')
Total_Topic = pd.read_csv('./LDA/Total_Topic.csv')

In [10]:
Total_Normal_S2V = pd.merge(Total_Normal, Total_S2V, left_index=True, right_index=True)
Total_Final = pd.merge(Total_Normal_S2V, Total_Topic, left_index=True, right_index=True)

In [11]:
Total_Final.to_csv('./Total_Final_Feature.csv', encoding='utf-8', index=False)

In [12]:
test_len = [7, 11, 5, 7, 8, 6, 10, 9, 8, 8, 11, 7, 6, 7, 11, 5, 5, 7, 10, 5, 7, 9, 7, 6, 8, 7, 9, 9, 5, 6, 7, 6, 5, 7, 6, 11, 6, 8, 6, 7, 7, 5, 6, 8, 6, 7, 6, 6, 7, 10]

In [13]:
start = end = 3340
for i in range( len(test_len) ) :
    end += test_len[i]
    print (start, end)
    test_i = Total_Final[start:end]
    test_i.to_csv('../MODEL/SVM_Data/TEST/' + 'test_' + str(i) + '.csv', encoding='utf-8', index = False)
    start += test_len[i]

(3340, 3347)
(3347, 3358)
(3358, 3363)
(3363, 3370)
(3370, 3378)
(3378, 3384)
(3384, 3394)
(3394, 3403)
(3403, 3411)
(3411, 3419)
(3419, 3430)
(3430, 3437)
(3437, 3443)
(3443, 3450)
(3450, 3461)
(3461, 3466)
(3466, 3471)
(3471, 3478)
(3478, 3488)
(3488, 3493)
(3493, 3500)
(3500, 3509)
(3509, 3516)
(3516, 3522)
(3522, 3530)
(3530, 3537)
(3537, 3546)
(3546, 3555)
(3555, 3560)
(3560, 3566)
(3566, 3573)
(3573, 3579)
(3579, 3584)
(3584, 3591)
(3591, 3597)
(3597, 3608)
(3608, 3614)
(3614, 3622)
(3622, 3628)
(3628, 3635)
(3635, 3642)
(3642, 3647)
(3647, 3653)
(3653, 3661)
(3661, 3667)
(3667, 3674)
(3674, 3680)
(3680, 3686)
(3686, 3693)
(3693, 3703)
